In [92]:
import numpy as np

In [102]:
def GetData():
    file = open("iris.data")
    data = file.readlines()
    data = [d.split(',') for d in data[:-1]]

    X = [d[:-1] for d in data]
    for i in range(len(X)):
        X[i] = [float(x) for x in X[i]]


    Y = [d[-1] for d in data]
    dic = {}
    for k,v in enumerate(set(Y)):
        dic[v]=k
    Y = [dic[v] for v in Y]
    
    X = np.array(X)
    Y = np.array(Y)
    Y = np.reshape(Y,(150,1))
    data = np.append(X,Y,1)
    setosa = data[0:50]
    versicolor = data[50:100]
    virginica = data[100:]
    np.random.shuffle(setosa)
    np.random.shuffle(versicolor)
    np.random.shuffle(virginica)
    train = np.concatenate((setosa[0:30],versicolor[0:30],virginica[0:30]),axis=0)
    val = np.concatenate((setosa[30:40],versicolor[30:40],virginica[30:40]),axis=0)
    test = np.concatenate((setosa[40:],versicolor[40:],virginica[40:]),axis=0)
    return train,val,test,dic


In [103]:
train,val,test,dic = GetData()

In [104]:
def euclidean_distance(a, b):
    distance = np.linalg.norm(a[:-1]-b[:-1])
    return distance

In [105]:
def predict_nearest_neighbors(train, test_row):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda t: t[1])
    return int(distances[0][0][4])

In [106]:
def predict_min_farthest_neighbors(train, test_row):
    
    setosa = train[train[:,4]==0];
    virginica = train[train[:,4]==2];
    versicolor = train[train[:,4]==1];
    
    a = b = c = -1
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        if train_row[4] == 0 and a < dist:
            a = dist;
        if train_row[4] == 2 and b < dist:
            b = dist;
        if train_row[4] == 1 and c < dist:
            c = dist;
    
    m = min(a,b,c)
    
    if m == a :
        return 0
    elif m == b :
        return 2
    else:
        return 1

In [107]:
def predict_nearest_centroid(train, test_row):
    
    setosa = train[train[:,4]==0];
    virginica = train[train[:,4]==2];
    versicolor = train[train[:,4]==1];
    
    centroid_setosa = np.mean(setosa,axis=0); 
    centroid_virginica = np.mean(virginica,axis=0); 
    centroid_versicolor = np.mean(versicolor,axis=0); 
    
    a = euclidean_distance(test_row, centroid_setosa)
    b = euclidean_distance(test_row, centroid_virginica)
    c = euclidean_distance(test_row, centroid_versicolor)
    
    m = min(a,b,c)
    
    if m == a :
        return 0
    elif m == b :
        return 2
    else:
        return 1

In [108]:
def predict(train, test_row):
    n = predict_nearest_neighbors(train,test_row);
    f = predict_min_farthest_neighbors(train,test_row);
    c = predict_nearest_centroid(train,test_row);
    vals = [n,f,c]
    pred = max(set(vals), key=vals.count)
    return vals,pred

In [109]:
def cal_accuracy(actual,predicted):
    x = actual == predicted;
    x = x.count(True)
    x = (x/len(actual))*100
    return x

In [110]:
def validate(train,target):
    pn = 0
    pf = 0
    pc = 0
    pp = 0
    #actual = 
    print('\033[1m' + ' Nearest  Farthest  Centroid         Predicted')
    print('\033[0m')
    for v in target:
        act = v[4]
        vals,pred = predict(train,v)
        
        if pred == act :
            pp += 1
        if vals[0] == act :
            pn += 1
        if vals[1] == act :
            pf += 1
        if vals[2] == act :
            pc += 1
        
        #print(' Actual : {} Predicted : {} Data : {} Neighbour : {}'.format(int(act),int(pred),v,neighbour))
        print("   {}         {}         {}                 {}  ".format(vals[0],vals[1],vals[2],pred)) 
    
    an = round((pn/len(target))*100,2);
    af = round((pf/len(target))*100,2);
    ac = round((pc/len(target))*100,2);
    ap = round((pp/len(target))*100,2);
    
    print('\033[1m')
    
    print(" {} %    {} %   {} %          {} %  ".format(an,af,ac,ap)) 
    

In [91]:
np.random.shuffle(train)
np.random.shuffle(val)
np.random.shuffle(test) 
validate(train,val)
print("\n\n")
validate(train,test)

 Nearest  Farthest  Centroid         Predicted

   1         1         1                 1  
   0         1         1                 1  
   2         2         2                 2  
   0         1         1                 1  
   1         1         1                 1  
   2         2         2                 2  
   1         1         1                 1  
   2         2         2                 2  
   2         2         2                 2  
   1         1         1                 1  
   1         1         1                 1  
   0         1         1                 1  
   2         2         2                 2  
   1         2         1                 1  
   2         2         2                 2  
   2         2         2                 2  
   2         2         2                 2  
   0         0         0                 0  
   1         1         1                 1  
   0         1         0                 0  
   0         0         0                 0  
   2   